In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in the data
df = pd.read_csv('dataset_mood_smartphone-2.csv')

# convert time column to datetime format
df['time'] = pd.to_datetime(df['time'])

# round time to nearest hour and group by time, person_id, and variable
df['time_hour'] = df['time'].dt.round('D')
grouped_df = df.groupby(['time_hour', 'id', 'variable']).sum().reset_index()

# use pivot_table to create columns for each variable
pivoted_df = grouped_df.pivot_table(index=['time_hour', 'id'], columns='variable', values='value')

pivoted_df

variable            activity  appCat.builtin  appCat.communication  \
time_hour  id                                                        
2014-02-17 AS14.02       NaN             NaN                   NaN   
           AS14.06       NaN             NaN                   NaN   
           AS14.09       NaN             NaN                   NaN   
           AS14.14       NaN             NaN                   NaN   
           AS14.16       NaN             NaN                   NaN   
...                      ...             ...                   ...   
2014-06-05 AS14.24  1.510372         843.588              4203.992   
2014-06-06 AS14.24  3.801124        1856.035              1358.551   
2014-06-07 AS14.24  5.349713        5314.422              1254.974   
2014-06-08 AS14.24  0.602705         686.255               739.230   
2014-06-09 AS14.24  0.250000             NaN                   NaN   

variable            appCat.entertainment  appCat.finance  appCat.game  \
time_hour  id                                                           
2014-02-17 AS14.02                   NaN             NaN          NaN   
           AS14.06                   NaN             NaN          NaN   
           AS14.09                   NaN             NaN          NaN   
           AS14.14                   NaN             NaN          NaN   
           AS14.16                   NaN             NaN          NaN   
...                                  ...             ...          ...   
2014-06-05 AS14.24                   NaN             NaN          NaN   
2014-06-06 AS14.24                   NaN             NaN          NaN   
2014-06-07 AS14.24                   NaN             NaN          NaN   
2014-06-08 AS14.24                   NaN             NaN          NaN   
2014-06-09 AS14.24                   NaN             NaN          NaN   

variable            appCat.office  appCat.other  appCat.social  appCat.travel  \
time_hour  id                                                                   
2014-02-17 AS14.02            NaN           NaN            NaN            NaN   
           AS14.06            NaN           NaN            NaN            NaN   
           AS14.09            NaN           NaN            NaN            NaN   
           AS14.14            NaN           NaN            NaN            NaN   
           AS14.16            NaN           NaN            NaN            NaN   
...                           ...           ...            ...            ...   
2014-06-05 AS14.24            NaN        43.519         87.047            NaN   
2014-06-06 AS14.24            NaN        41.542        153.855        225.737   
2014-06-07 AS14.24            NaN        29.194            NaN            NaN   
2014-06-08 AS14.24            NaN        24.283        297.610            NaN   
2014-06-09 AS14.24            NaN           NaN            NaN            NaN   

variable            appCat.unknown  appCat.utilities  appCat.weather  call  \
time_hour  id                                                                
2014-02-17 AS14.02             NaN               NaN             NaN   1.0   
           AS14.06             NaN               NaN             NaN   2.0   
           AS14.09             NaN               NaN             NaN   2.0   
           AS14.14             NaN               NaN             NaN   NaN   
           AS14.16             NaN               NaN             NaN   1.0   
...                            ...               ...             ...   ...   
2014-06-05 AS14.24             NaN               NaN             NaN   3.0   
2014-06-06 AS14.24             NaN             2.072             NaN  11.0   
2014-06-07 AS14.24             NaN               NaN             NaN  14.0   
2014-06-08 AS14.24             NaN               NaN             NaN   NaN   
2014-06-09 AS14.24             NaN               NaN             NaN   NaN   

variable            circumplex.arousal  circumplex.valence  mood      